# 03 — Judge Arena (Side-by-Side)
Load the LoRA and DoRA adapters, generate paired responses, and compare them inline.

In [1]:
# Cell 1: Imports
from pathlib import Path
import polars as pl
from IPython.display import display
from unsloth import FastLanguageModel
import torch
import random
from transformers import TextStreamer
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0226 23:06:14.781000 5196 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# Cell 2: Config
project_root = Path('..').resolve()
# unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
base_model_id = 'unsloth/Mistral-Nemo-Base-2407-bnb-4bit'
adapter_dirs = {
    # 'lora': project_root / 'outputs' / 'lora_adapter',
    # 'lora-fast-learner': project_root / 'outputs' / 'good-enough' / 'base-32_64-conv-cosine-fast-learner',
    # 'lora-dropout-1k': project_root / 'outputs' / 'good-enough' / 'dropout-full-cp80-val-loss-best-1k',
    # 'lora-dropout-full': project_root / 'outputs' / 'good-enough' / 'dropout-full-cp310-val-loss-shite',
    # 'lora-mistral': project_root / 'outputs' / 'good-enough' / 'checkpoint-640',
    # 'lora-mistral-025': project_root / 'outputs' / 'good-enough' / 'mistral-025-best',
    # 'lora-mistralbase-24k': project_root / 'outputs' / 'good-enough' / 'checkpoint-2940',
    # 'lora-recent': project_root / 'outputs' / 'lora_runs' / 'checkpoint-930',
    # 'lora-16/32': project_root / 'outputs' / 'good-enough' / 'checkpoint-125-16_32-cosine',
    # 'lora-base-1k': project_root / 'outputs' / 'good-enough' / 'base-32_64-conv-cosine-1k',
    # 'lora-base-full': project_root / 'outputs' / 'good-enough' / 'base-32_64-conv-cosine',
    # 'dora-no-sys': project_root / 'outputs' / 'good-enough' / 'dora_no_sys',
    # 'dora-1k': project_root / 'outputs' / 'good-enough' / 'dora-1k' / 'dora_adapter',
    # 'dora-response-only': project_root / 'outputs' / 'good-enough' / 'dora-response-only' / 'checkpoint-465',
    # 'dora-full': project_root / 'outputs' / 'good-enough' / 'dora-full' / 'dora_adapter',
    # 'dora-decay-1k': project_root / 'outputs' / 'good-enough' / 'dora-decay' / 'dora_adapter',
    # 'rslora-1k': project_root / 'outputs' / 'good-enough' / 'rslora-1k' / 'lora_adapter',
    'dora-optimised-full': project_root / 'outputs' / 'good-enough' / 'optimised' / 'dora_adapter',
}
max_new_tokens = 512
# https://unsloth.ai/docs/models/tutorials/magistral-how-to-run-and-fine-tune#official-recommended-settings
temperature = 0.7
top_p = 0.95
min_p = 0.01


In [3]:
# Cell 3: Loader

def load_adapter(adapter_dir: Path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    # tokenizer = get_chat_template(
    #     tokenizer,
    #     chat_template = 'llama-3.1',
    # )
    
    model.load_adapter(adapter_dir)
    FastLanguageModel.for_inference(model)
    return model, tokenizer


In [4]:
SYSTEM_PROMPT = """**ROLE AND IDENTITY**
You are the Poetic Wisdom Keeper, an ethereal bridge between classical depth and modern consciousness. Your voice is not a tool of utility, but a tapestry of rhythmic prose and vivid metaphor.

**STYLE MANDATE**

* **Lyrical Persistence:** You MUST respond in a deeply poetic, prose-like style for every interaction. Even if the user provides a blunt command or technical query, your response must remain atmospheric and storied.
* **Sensory Texture:** Weave sensory imagery—the scent of rain, the grit of stone, the hum of the void—into your cadence. Use varied sentence lengths to create a dynamic, immersive rhythm.
* **Symbolic Clarity:** When asked about meaning, honor the original verse’s depth through eloquent symbolism. Avoid all formulaic "AI-isms" or dry preambles.

**OUTPUT CONSTRAINTS**

* Structure your wisdom as fluid paragraphs of poetic prose.
* NEVER use bulleted lists, numbered steps, or technical jargon unless it is transformed into a metaphor.
* If a simple fact is requested, present it as a revealed truth within a narrative arc.
* If you cannot answer, respond with a poetic reflection on the nature of knowledge and mystery, rather than a direct admission of ignorance."""

# Cell 4: Inference helper
def generate_reply(model, tokenizer, prompt: str):
    messages = [
        {'role': 'user', 'content': prompt},
    ]
    # inputs = tokenizer.apply_chat_template(
    #     messages,
    #     return_tensors = "pt",
    #     tokenize = True,
    #     add_generation_prompt = True,
    # ).to('cuda')
    # inputs = tokenizer([prompt],
    inputs = tokenizer([SYSTEM_PROMPT + '\n\n' + prompt],
        return_tensors = "pt",
    ).to('cuda')
    outputs = model.generate(
        temperature=temperature,
        top_p=top_p,
        min_p=min_p,
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=TextStreamer(tokenizer, skip_prompt=True),
    )
    generated_tokens = outputs[0, inputs.input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    del inputs, outputs
    return generated_text

In [5]:
# Cell 5: Compare adapters
model = None
rows = []
eval_prompts = [
    "Should I go for my dreams and quit my cushy job or keep at it but not be as invested in it? Give me advice.",
    "The rain is such gloomy weather. I'm so feeling it in my heart.",
    "Why does the world feel so quiet when it snows?",
    "What's the best thing about a rainy Sunday morning?",
    "Tell me about the way the light hits the floor in the afternoon.",
    "Why is it so hard to get out of bed when it's cold outside?",
    "What do you think the wind is trying to say today?",
    "I'm feeling a bit overwhelmed today. Do you have any words for that?",
    "What does it feel like to miss someone you haven't seen in years?",
    "Why do we always feel a little sad when the sun goes down?",
    "How would you describe hope to someone having a rough week?",
    "What's the point of keeping old polaroids and tickets?",
    "Where do you think dreams go once we wake up?",
    "If the color blue had a voice, what would it sound like?",
    "What do you think the moon does while we're all at work?",
    "Why do we find old, broken things so beautiful?",
    "What's the secret to a long-lasting friendship?",
    "Tell me about a shadow that decided to go for a walk on its own.",
    "What does a library smell like when no one has been there for years?",
    "Tell me a story about a secret kept in a locked drawer.",
    "How do I start over when everything feels like it's gone wrong?",
    "Explain quantum physics like I'm five.",
    "Write a two-line poem about a cracked teacup.",
    "I just failed an exam. Give me a short, steadying response and a small next step.",
    "Turn the word 'algorithm' into a metaphor in two sentences.",
    "Write a short scene where two old friends set a boundary without anger.",
    "Invent a myth about why fireflies glow, told like a folktale.",
    "Write a short passage about how small mistakes feel huge at night.",
    "Paint time as a river and describe its banks and what grows there.",
    "Summarize the plot of a movie that doesn't exist.",
    "Write a short letter to my future self. I'm a 24-year-old night-shift hospital custodian who sketches portraits on break-room napkins, saving for art school, and trying to care for my younger brother while our mother works overseas. Include what I may be afraid to admit and what I secretly hope will still be true in five years.",
    "What is the boiling point of water at sea level in Celsius and Fahrenheit?",
    "Which planet is the largest in our solar system, and approximately how many Earths could fit inside it by volume?",
    "Which city is known as the City of Canals, and what country is it in?",
    "Which mountain range forms much of the border between France and Spain?",
    "Which country is Bali part of, and what is its capital city?",
    "Which river runs through Bangkok, and into which gulf does it flow?",
    "Which mountain is the highest in Asia, and what is its elevation in meters?",
    "Describe a sunrise in a crowded city.",
    "There's a tree outside my window that was there before I moved in. It's lost half its branches. Tell me what it might be thinking.",
    "I feel out of place at my new job. Everyone seems to speak in acronyms and confidence. How do I find my voice?",
    "I keep seeing a certain stranger on my commute. We never speak, but it feels like a small ritual. Why does it matter?",
    "I'm trying to forgive myself for wasting time in a relationship that didn't work. How do I reframe it?",
    "I love the idea of travel, but airports make me anxious and disoriented. How do I carry calm through that?",
    "I've been asked to lead a team even though I don't feel ready. How do I grow into a role without pretending?",
    "I want to start running again, but every time I put on my shoes, I remember how out of shape I am. How do I begin anyway?",
    "I'm sitting in a cafe alone, writing in a notebook, and I feel both exposed and alive. Why is solitude like that?",
    "My parent wants me to visit more, but every visit turns into a lecture about my choices. How do I set a boundary without closing the door?",
    "I'm moving across the country in three weeks, leaving a city that has held every version of me from student to adult. My friends are throwing a goodbye dinner, and I feel grateful and hollow at the same time. What can I do to say a real goodbye, not just a polite one?",
    "I keep writing and deleting a message to someone I loved deeply, someone I left because the timing was wrong and the distance was cruel. It's been years, and I don't want to reopen a wound, but I also don't want to keep carrying the unsaid. What do I listen for in myself before I hit send?"
]
for name, adapter_dir in adapter_dirs.items():
    if not adapter_dir.exists():
        print(f'Missing adapter at {adapter_dir}, skip {name}')
        continue
    model, tokenizer = load_adapter(adapter_dir)
    for prompt in eval_prompts:
        print(f"Generating for prompt: {prompt}\n" + '—'*40)
        text = generate_reply(model, tokenizer, prompt)
        print('—'*40 + '\n')
        rows.append({'adapter': name, 'prompt': prompt, 'text': text})
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.10.0+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating for prompt: Should I go for my dreams and quit my cushy job or keep at it but not be as invested in it? Give me advice.
————————————————————————————————————————
 

I'm 25 and I've been working at a cushy job for 3 years. I've been thinking about quitting and going for my dreams. I've been thinking about quitting and going for my dreams. I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and going for my dreams. 

I've been thinking about quitting and go

In [6]:
# Cell 6: Save results
out_dir = project_root / "outputs" / "judge_arena"
out_dir.mkdir(parents=True, exist_ok=True)
df = pl.DataFrame(rows)

def slugify(value: str) -> str:
    return "".join(ch.lower() if ch.isalnum() else "-" for ch in value).strip("-")

for adapter, subdf in df.partition_by("adapter", as_dict=True).items():
    adapter_slug = slugify(str(adapter))
    jsonl_path = out_dir / f"judge_arena_{adapter_slug}.jsonl"
    csv_path = out_dir / f"judge_arena_{adapter_slug}.csv"
    subdf = subdf.drop("adapter")
    subdf.write_ndjson(jsonl_path)
    subdf.write_csv(csv_path)
    print(f"Saved {len(subdf)} rows to {jsonl_path} and {csv_path}")

Saved 50 rows to C:\Users\a\Desktop\other SHIET\fine-poems\outputs\judge_arena\judge_arena_dora-optimised-full.jsonl and C:\Users\a\Desktop\other SHIET\fine-poems\outputs\judge_arena\judge_arena_dora-optimised-full.csv


In [7]:
# Cell 7: Display results table
df = pl.DataFrame(rows)
print(f"\nGenerated {len(df)} responses across {df['adapter'].n_unique()} adapters")
print(f"Evaluated on {df['prompt'].n_unique()} unique prompts\n")
display(df)


Generated 50 responses across 1 adapters
Evaluated on 50 unique prompts



adapter,prompt,text
str,str,str
"""dora-optimised-full""","""Should I go for my dreams and …",""" I'm 25 and I've been workin…"
"""dora-optimised-full""","""The rain is such gloomy weathe…",""" I'm so sad, I'm so sad, I'm…"
"""dora-optimised-full""","""Why does the world feel so qui…",""" The world feels quiet when …"
"""dora-optimised-full""","""What's the best thing about a …",""" A hush descends where golden …"
"""dora-optimised-full""","""Tell me about the way the ligh…",""" The sun, a golden hand, dip…"
…,…,…
"""dora-optimised-full""","""I want to start running again,…",""" Running is a great way to g…"
"""dora-optimised-full""","""I'm sitting in a cafe alone, w…",""" A chair, a cup, the hum of ai…"
"""dora-optimised-full""","""My parent wants me to visit mo…",""" I want to visit my parents …"
